In [0]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision
import torch.optim as optim

In [0]:
if torch.cuda.is_available :
  device = torch.device('cuda:0')
else :
  device = torch.device('cpu')

# print(device)

In [3]:
train_data = torchvision.datasets.MNIST(root = './data', train = True, download = True, transform = transforms.ToTensor())
test_data = torchvision.datasets.MNIST(root = './data', train = False, download = True, transform = transforms.ToTensor())

0it [00:00, ?it/s]

9920512it [00:02, 3554476.14it/s]                             


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:00, 56189.55it/s]                           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:01, 943846.16it/s]                             
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 21162.65it/s]            

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = 100, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = 100, shuffle = False)

In [0]:
class ConvNN(nn.Module):
    def __init__(self):
        super(ConvNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 20, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(20, 40, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Sequential(
          nn.Linear(7 * 7 * 40, 512),
          nn.Sigmoid(),
          nn.Linear(512, 128),
          nn.Sigmoid(),
          nn.Linear(128, 10))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [0]:
epochs = 10
alpha = 0.0001

In [0]:
cnn = ConvNN().to(device)

In [0]:
criterion = nn.CrossEntropyLoss()
opt = optim.Adam(cnn.parameters(), lr = alpha)

In [9]:
total_step = len(train_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = cnn.forward(images)
        loss = criterion(outputs, labels)
        
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        
    print('Epoch : {} / {}  |  Loss : {:.6f}'.format(epoch + 1, epochs, loss))

Epoch : 1 / 10  |  Loss : 0.369924
Epoch : 2 / 10  |  Loss : 0.137392
Epoch : 3 / 10  |  Loss : 0.068457
Epoch : 4 / 10  |  Loss : 0.049072
Epoch : 5 / 10  |  Loss : 0.060253
Epoch : 6 / 10  |  Loss : 0.020193
Epoch : 7 / 10  |  Loss : 0.050488
Epoch : 8 / 10  |  Loss : 0.013504
Epoch : 9 / 10  |  Loss : 0.023681
Epoch : 10 / 10  |  Loss : 0.029154


In [10]:
cnn.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = cnn(images)
        pred = torch.argmax(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()

    print('correct : {}'.format(correct))
    print('total : {}'.format(total))
    print('accuracy : {}'.format(100 * correct / total))


correct : 9914
total : 10000
accuracy : 99.14
